In [158]:
import sympy as sp
sp.init_printing(use_latex='mathjax')

from sympy import symbols, sin, cos, sqrt, asin, atan


In [159]:
# Actuation

theta = symbols('theta')

# passive joint 
psi = symbols('psi')


In [160]:

# Constants defining geometry.
theta0 = symbols('theta0')

shank, p1, p2, p3 = symbols('shank p1 p2 p3')



In [161]:

k1 = shank / p1
k2 = shank / p3
k3 = (shank**2 + p1**2 + p3**2 - p2**2 ) / (2 * p1 * p3)

cT = cos(theta)
sT = sin(theta)

# precompute the stuff above
k1, k2, k3 = symbols('k1 k2 k3')


A = k1 * cT + k2 + k3 + cT # C.34
B = -2*sT # C.35
C = k1 * cT - k2 + k3 - cT # C.36

psi = 2 * atan((-B + sqrt(B * B - 4 * A * C)) / (2 * A)) #  C.39

psi_inner = (-B + sqrt(B * B - 4 * A * C)) / (2 * A)

psi = sp.simplify(psi)


from sympy import Function
psi = Function('psi')(theta)

In [162]:
psi

ψ(θ)

In [163]:

from sympy.vector import CoordSys3D, AxisOrienter
from sympy import Matrix

B = CoordSys3D('base')

# zAxis = Vector(0,0, 1)

orienter_psi = AxisOrienter(-psi, B.k )

T_passive = B.orient_new('passive', (orienter_psi, ))

T_active_base = T_passive.locate_new('active_base', shank * T_passive.i )

T_active_base



active_base

In [164]:

orienter_theta = AxisOrienter(theta, B.k )

T_active = T_active_base.orient_new('active', (orienter_theta, ))

T_active

active

In [165]:
# express the end effector in the active joint coordinate system

x, y,z , alpha, beta, gamma = symbols("x y z alpha beta gamma")

from sympy.vector import BodyOrienter, Point
euler = BodyOrienter(alpha, beta, gamma, rot_order="XYZ")

P = T_active.orient_new("EEF", euler, location=(T_active.i * x + T_active.j * y + T_active.k * z))



In [166]:
pos_vect = P.origin.position_wrt(B)
p = pos_vect.to_matrix(B)

o = P.rotation_matrix(B)

In [167]:
from sympy import diff

dp_dtheta = diff(p, theta)
dp_dtheta

⎡                   d            ⎛                 d                          
⎢ - shank⋅sin(ψ(θ))⋅──(ψ(θ)) + x⋅⎜sin(θ)⋅cos(ψ(θ))⋅──(ψ(θ)) - sin(θ)⋅cos(ψ(θ))
⎢                   dθ           ⎝                 dθ                         
⎢                                                                             
⎢                  d            ⎛                   d                         
⎢- shank⋅cos(ψ(θ))⋅──(ψ(θ)) + x⋅⎜- sin(θ)⋅sin(ψ(θ))⋅──(ψ(θ)) + sin(θ)⋅sin(ψ(θ)
⎢                  dθ           ⎝                   dθ                        
⎢                                                                             
⎣                                                                             

                    d                          ⎞     ⎛                 d      
 - sin(ψ(θ))⋅cos(θ)⋅──(ψ(θ)) + sin(ψ(θ))⋅cos(θ)⎟ + y⋅⎜sin(θ)⋅sin(ψ(θ))⋅──(ψ(θ)
                    dθ                         ⎠     ⎝                 dθ     
                                                   

In [168]:
o

⎡ (-(sin(α)⋅sin(β)⋅cos(γ) + sin(γ)⋅cos(α))⋅sin(θ) + cos(β)⋅cos(γ)⋅cos(θ))⋅cos(
⎢                                                                             
⎢(-(-sin(α)⋅sin(β)⋅sin(γ) + cos(α)⋅cos(γ))⋅sin(θ) - sin(γ)⋅cos(β)⋅cos(θ))⋅cos(
⎢                                                                             
⎣                                 (sin(α)⋅sin(θ)⋅cos(β) + sin(β)⋅cos(θ))⋅cos(ψ

ψ(θ)) + ((sin(α)⋅sin(β)⋅cos(γ) + sin(γ)⋅cos(α))⋅cos(θ) + sin(θ)⋅cos(β)⋅cos(γ))
                                                                              
ψ(θ)) + ((-sin(α)⋅sin(β)⋅sin(γ) + cos(α)⋅cos(γ))⋅cos(θ) - sin(γ)⋅sin(θ)⋅cos(β)
                                                                              
(θ)) + (-sin(α)⋅cos(β)⋅cos(θ) + sin(β)⋅sin(θ))⋅sin(ψ(θ))                      

⋅sin(ψ(θ))    -(-(sin(α)⋅sin(β)⋅cos(γ) + sin(γ)⋅cos(α))⋅sin(θ) + cos(β)⋅cos(γ)
                                                                              
)⋅sin(ψ(θ))  -(-(-sin(α)⋅sin(β)⋅sin(γ) + cos(α)⋅co

In [169]:
# dp_dtheta_simpl = dp_dtheta.simplify()

# dx_dtheta = sp.simplify(dp_dtheta[0])

In [176]:
from sympy import ccode #, cpp_generator



cpp_code = ccode(dp_dtheta[0], assign_to="dx_dtheta", standard="c11")

with open("/tmp/expressions.cpp", "w") as f:
  f.write(cpp_code)

In [179]:
# from sympy.codegen.rewriting import optimize, optims_c99

# f = optimize(dp_dtheta[0], optims_c99)

# f

dp_dtheta[0]



                  d            ⎛                 d                            
- shank⋅sin(ψ(θ))⋅──(ψ(θ)) + x⋅⎜sin(θ)⋅cos(ψ(θ))⋅──(ψ(θ)) - sin(θ)⋅cos(ψ(θ)) -
                  dθ           ⎝                 dθ                           

                  d                          ⎞     ⎛                 d        
 sin(ψ(θ))⋅cos(θ)⋅──(ψ(θ)) + sin(ψ(θ))⋅cos(θ)⎟ + y⋅⎜sin(θ)⋅sin(ψ(θ))⋅──(ψ(θ)) 
                  dθ                         ⎠     ⎝                 dθ       

                                      d                          ⎞
- sin(θ)⋅sin(ψ(θ)) + cos(θ)⋅cos(ψ(θ))⋅──(ψ(θ)) - cos(θ)⋅cos(ψ(θ))⎟
                                      dθ                         ⎠

In [180]:
dp_dtheta[0].simplify()

                  d            ⎛d           ⎞                   ⎛d           ⎞
- shank⋅sin(ψ(θ))⋅──(ψ(θ)) + x⋅⎜──(ψ(θ)) - 1⎟⋅sin(θ - ψ(θ)) + y⋅⎜──(ψ(θ)) - 1⎟
                  dθ           ⎝dθ          ⎠                   ⎝dθ          ⎠

              
⋅cos(θ - ψ(θ))
              

In [181]:
dp_dtheta[1].simplify()

                  d            ⎛d           ⎞                   ⎛d           ⎞
- shank⋅cos(ψ(θ))⋅──(ψ(θ)) - x⋅⎜──(ψ(θ)) - 1⎟⋅cos(θ - ψ(θ)) + y⋅⎜──(ψ(θ)) - 1⎟
                  dθ           ⎝dθ          ⎠                   ⎝dθ          ⎠

              
⋅sin(θ - ψ(θ))
              

In [182]:
dp_dtheta[2].simplify()

0

In [175]:
psi_precompute = symbols("psi_precompute")
dpsi_dtheta_precompute = symbols("dpsi_dtheta_precompute")
psi_inner_precompute = symbols("psi_inner_precompute")

dpsi_inner = diff(psi_inner, theta)
dpsi_inner_precompute = symbols("dpsi_inner")

dpsi_dtheta = diff(psi, theta)

s = dp_dtheta[0].replace(psi, psi_precompute)
s = s.replace(dpsi_dtheta, dpsi_dtheta_precompute)
s = s.replace(psi_inner, psi_inner_precompute)
s = s.replace(dpsi_inner, dpsi_inner_precompute)

dpsi_dtheta

d       
──(ψ(θ))
dθ      

In [173]:
dpsi_dtheta

d       
──(ψ(θ))
dθ      

In [174]:
ff = s / dpsi_dtheta

ff.simplify()

x⋅sin(ψ_precompute - θ) - y⋅cos(ψ_precompute - θ)
─────────────────────────────────────────────────
                     d                           
                     ──(ψ(θ))                    
                     dθ                          